In [ ]:
import numpy as np
import random
from scipy.optimize import minimize,NonlinearConstraint,Bounds,differential_evolution

In [ ]:
#R-2*0.0095 per km
#line data        from, to, resistance
lines = np.array([[1, 2, 153.28],
                  [2, 3, 153.28],
                  [3, 4, 71.53],
                  [4, 1, 97.54],
                  [1, 3, 119.224]
                  ])


# Base quantities
base_voltage = 400  # 400kV
base_power = 784.8  # 784.8MW
base_current = 1962  # 1962A
base_resistance = base_voltage*1000 / base_current  #


dist = np.array([70,70,150,110,90])
r=(2*dist[:]*0.0095)/(base_resistance)
print(r)
r=1/r
print(r)

# Network Structure
num_lines = lines.shape[0]
num_buses = int(np.max(lines[:, :2]))

# Max-Min Constraints
I_max = np.array([1962] * num_lines) / base_current
P_gen = np.array([1.49, 0, 0, 0.5])
V_min = 0.95
V_max = 1.05
Vs_max = 0.05
on_shore = [2, 3]
kdroop=[21.739,21.739]

[0.00652365 0.00652365 0.01397925 0.01025145 0.00838755]
[153.28841983 153.28841983  71.53459592  97.54717625 119.22432653]


# Contingency Ranking

In [ ]:
def create_adjacency_matrix(lines):
    num_buses = int(np.max(lines[:, :2]))
    adj = np.zeros(shape=(num_buses, num_buses))
    for line in lines:
        adj[int(line[0]) - 1][int(line[1]) - 1] = line[2]
        adj[int(line[1]) - 1][int(line[0]) - 1] = line[2]
    return adj

def get_ybus_matrix(lines):
    num_buses = int(np.max(lines[:, :2]))
    adj = create_adjacency_matrix(lines)
    y_bus = np.zeros(shape=(num_buses, num_buses))
    for i in range(num_buses):
        for j in range(num_buses):
            y_bus[i][j] = -1 * adj[i][j] if i != j else np.sum(adj[i])
    return y_bus


def power_check(V_bus):
    V_bus = [V_bus[:-len(on_shore)],V_bus[-len(on_shore):]]
    power_onshore = np.zeros((num_buses,1))
    for i in range(len(on_shore)):
      bus=on_shore[i]-1
      power_onshore[bus][0]=-kdroop[i]*(V_bus[0][bus]-V_bus[1][i])
    return np.array([0,1.08695,1.08695,0])-np.abs(power_onshore.flatten())


def current_check(V_bus):
    I = []
    I_max = np.array([1962] * lines.shape[0]) / base_current
    for line in lines:
        I.append((V_bus[int(line[1]) - 1] - V_bus[int(line[0]) - 1]) * line[2])
    I = np.array(I)
    return I_max-np.abs(I)

def performance_index(V_bus):
    V_bus = [V_bus[:-len(on_shore)],V_bus[-len(on_shore):]]
    I = []
    I_max = np.array([1962] * lines.shape[0]) / base_current
    for line in lines:
        I.append((V_bus[0][int(line[1]) - 1] - V_bus[0][int(line[0]) - 1]) * line[2])
    I = np.array(I)
    I_norm = I / I_max
    return np.sum(I_norm**4)/ 4

def get_tripped_system(n):
    global lines
    lines= np.delete(lines,n-1,axis=0)
    return lines

# power balance
def power_balance(V_bus):
    # V_bus[0] is basically network side voltages and V_bus[1] is on shore voltage vector
    V_bus = [V_bus[:-len(on_shore)],V_bus[-len(on_shore):]]
    P = np.diag(V_bus[0].reshape((-1,1))[:, 0]) @ Y_bus @ V_bus[0]
    power_onshore = np.zeros((num_buses,1))
    for i in range(len(on_shore)):
      bus=on_shore[i]-1
      power_onshore[bus][0]=-kdroop[i]*(V_bus[0][bus]-V_bus[1][i])
    return P-P_gen-power_onshore.flatten()

def get_opf_results(trip=False):
    condition1 = {'type': 'eq', 'fun': lambda V_bus: power_balance(V_bus).flatten()}
    condition3 = {'type': 'ineq', 'fun': lambda V_bus: power_check(V_bus).flatten()}

    conditions=[]

    global lines
    global Y_bus

    if(trip!=False):
      lines= get_tripped_system(trip)
      conditions = [condition1,condition3]
    else:
      condition2 = {'type': 'ineq', 'fun': lambda V_bus: current_check(V_bus).flatten()}
      conditions = [condition1, condition2,condition3]

    Y_bus = get_ybus_matrix(lines)

    lower_bounds=[V_min]*(num_buses+len(on_shore))
    upper_bounds=[V_max]*(num_buses+len(on_shore))

    bounds=Bounds(lb=lower_bounds, ub=upper_bounds)

    x0 = np.array([1.05 for i in range(num_buses+len(on_shore))])

    sol = minimize(performance_index, x0, method='trust-constr', bounds=bounds, constraints=conditions, options={'maxiter':2500})

    V_bus = np.array(sol.x[:-1*len(on_shore)])

    return sol

In [ ]:
lines = np.array([[1, 2, 153.28],
                  [2, 3, 153.28],
                  [3, 4, 71.53],
                  [4, 1, 97.54],
                  [1, 3, 119.224]
                  ])
get_opf_results()

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


           message: `gtol` termination condition is satisfied.
           success: True
            status: 1
               fun: 0.17178932505721903
                 x: [ 1.049e+00  1.044e+00  1.044e+00  1.050e+00  1.003e+00
                      9.941e-01]
               nit: 43
              nfev: 231
              njev: 33
              nhev: 0
          cg_niter: 42
      cg_stop_cond: 4
              grad: [ 1.207e+02 -8.987e+01 -3.620e+01  5.390e+00  0.000e+00
                      0.000e+00]
   lagrangian_grad: [-6.018e-14 -1.121e-15  2.423e-14  1.132e-14  6.840e-15
                      5.688e-15]
            constr: [array([-4.902e-09,  2.438e-09, -3.253e-09, -6.027e-09]), array([ 1.630e-01,  9.820e-01,  5.780e-01,  9.458e-01,
                            3.630e-01]), array([ 0.000e+00,  1.944e-01,  8.013e-08,  0.000e+00]), array([ 1.049e+00,  1.044e+00,  1.044e+00,  1.050e+00,
                            1.003e+00,  9.941e-01])]
               jac: [array([[ 3.898e+02, -1.609

In [ ]:
x=get_opf_results(trip=1).x
x

array([1.05000001, 1.0338964 , 1.03944364, 1.04835478, 0.99345741,
       0.98944364])

In [ ]:
# Trip Line 1
num_lines=5
pi=[]
for i in range(1,num_lines+1):
  lines = np.array([[1, 2, 153.28],
                  [2, 3, 153.28],
                  [3, 4, 71.53],
                  [4, 1, 97.54],
                  [1, 3, 119.224]
                  ])
  ans=get_opf_results(trip=i).fun
  pi.append((ans,i))
  print("Tripping Line :", i, "PI:", ans)

Tripping Line : 1 PI: 0.7993761677670248
Tripping Line : 2 PI: 0.1790526229821357
Tripping Line : 3 PI: 0.4295361171028981
Tripping Line : 4 PI: 0.15739505751080524
Tripping Line : 5 PI: 0.6605290332803796


In [ ]:
pi.sort()
line_to_trip=pi[-1][1]

# Sensitivity Analysis

In [ ]:
lines = np.array([[1, 2, 153.28],
                  [2, 3, 153.28],
                  [3, 4, 71.53],
                  [1, 4, 97.54],
                  [1, 3, 119.224]
                  ])
V_bus = get_opf_results(trip=1).x
I=[0]*num_lines
c=1
for line in lines:
    I[c]=(V_bus[int(line[1]) - 1] - V_bus[int(line[0]) - 1]) * line[2]
    c+=1
line_currents = np.array(I)

In [ ]:
print("Bus Voltages :")
print(V_bus[:-2])
print("Control Voltages :")
print(V_bus[:-2])
print("line currents :")
print(line_currents)

Bus Voltages :
[1.05000001 1.0338964  1.03944364 1.04835478]
Control Voltages :
[1.05000001 1.0338964  1.03944364 1.04835478]
line currents :
[ 0.          0.85028173  0.63741345 -0.16047568 -1.25857193]


In [ ]:
candidate_lines=[(2,3),(2,5),(3,5),(3,4),(4,5)]

In [ ]:
D=[0]*len(candidate_lines)
c=0
print("Dnull values :")
for i,j in candidate_lines:
  D[c]=np.abs(line_currents[i-1])/(np.abs(line_currents[i-1])+np.abs(line_currents[j-1]))
  print(D[c])
  c+=1

Dnull values :
0.5715429764346314
0.4031961764108761
0.33619112112407823
0.7988747175822843
0.11308688678617519


In [ ]:
def get_pfc_power_der(i,j,k,Dnull,der):
  # Onshore Power Derivatives
  droop=np.array([0,21.73,21.73,0])
  onshore_der = -droop*der[:-1]

  # Offshore Power Derivatives
  offshore_der = np.array([0]*num_buses)

  # Injected Power Derivatives
  inj_der = np.array([0]*num_buses,dtype=np.float32)
  inj_der[i-1]=Y_bus[i-1][j-1]*(1-Dnull)*V_bus[i-1]*der[-1] - Y_bus[i-1][k-1]*Dnull*V_bus[i-1]*der[-1]
  inj_der[j-1]=-1*Y_bus[i-1][j-1]*(1-Dnull)*V_bus[j-1]*der[-1]
  inj_der[k-1]=Y_bus[i-1][k-1]*Dnull*V_bus[k-1]*der[-1]

  power_derivative = offshore_der + onshore_der + inj_der

  return power_derivative

import sympy as sp
def get_pfc_derivative(i,j,k,dnull):
  D2 = sp.symbols('D2')

  f1 = (D2*Y_bus[i-1][k-1]-(1 - D2)*Y_bus[i-1][j-1])/ (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])
  f2 = ((1- D2) * Y_bus[i-1][j-1]) / (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])
  f3 = (-D2*Y_bus[i-1][k-1]) / (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])


  derivative1 = sp.diff(f1, D2)
  derivative2 = sp.diff(f2, D2)
  derivative3 = sp.diff(f3, D2)

  der_value1 = derivative1.subs(D2, dnull)
  der_value2 = derivative2.subs(D2, dnull)
  der_value3 = derivative3.subs(D2, dnull)

  return np.array([der_value1,der_value2,der_value3])

def system_equations(i,j,k,Dnull,der):
    D2=Dnull
    f1 = (D2*Y_bus[i-1][k-1]-(1 - D2)*Y_bus[i-1][j-1])/ (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])
    f2 = ((1- D2) * Y_bus[i-1][j-1]) / (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])
    f3 = (-D2*Y_bus[i-1][k-1]) / (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])

    # Calculate derivatives and power balance
    pfc_derivative = np.array([f1,f2,f3])@np.array([der[i-1],der[j-1],der[k-1]]).T + get_pfc_derivative(i,j,k,Dnull)@np.array([V_bus[i-1],V_bus[j-1],V_bus[k-1]]).T - der[-1]
    power_balance_derivative=np.diag(der[:-1])@Y_bus@V_bus[:-2] + np.diag(V_bus[:-2])@Y_bus@der[:-1]-get_pfc_power_der(i,j,k,Dnull,der)

    # Return the array of residuals
    arr = np.concatenate((power_balance_derivative,np.array([pfc_derivative])))
    arr = np.array(list(map(float,arr)))
    return arr[0],arr[1],arr[2],arr[3],arr[4]


def get_sensitivity(l1,l2,i,j,k,Dnull):
  D2=Dnull
  f1 = (D2*Y_bus[i-1][k-1]-(1 - D2)*Y_bus[i-1][j-1])/ (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])
  f2 = ((1- D2) * Y_bus[i-1][j-1]) / (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])
  f3 = (-D2*Y_bus[i-1][k-1]) / (((1 - D2)**2) * Y_bus[i-1][j-1] + (D2**2) * Y_bus[i-1][k-1])

  from scipy.optimize import root

  # Initial guess for the solution vector der
  initial_guess = np.zeros(num_buses + 1)

  # Use the Newton-Raphson method to solve the system of equations
  result = root(lambda x :system_equations(i,j,k,Dnull,x), initial_guess, method='lm')

  # # Get the final solution
  voltage_derivatives = result.x

  der=voltage_derivatives
  pfc_derivative=np.array([f1,f2,f3])@np.array([der[i-1],der[j-1],der[k-1]]).T + get_pfc_derivative(i,j,k,Dnull)@np.array((V_bus[i-1],V_bus[j-1],V_bus[k-1])).T - der[-1]
  power_balance_derivative=np.diag(der[:-1])@Y_bus@V_bus[:-2] + np.diag(V_bus[:-2])@Y_bus@der[:-1]-get_pfc_power_der(i,j,k,Dnull,der)

  # print(pfc_derivative,power_balance_derivative)
  # print(der)
  c=1

  sensitivity=0
  for line in lines:
    if(c+1==l1):
      sensitivity+=(I[c]**3)*((voltage_derivatives[int(line[1]) - 1] - voltage_derivatives[int(line[0]) - 1]+ (-Dnull+1)*voltage_derivatives[-1]) * line[2])
    elif(c+1==l2):
      sensitivity+=(I[c]**3)*((voltage_derivatives[int(line[1]) - 1] - voltage_derivatives[int(line[0]) - 1]+ (-Dnull)*voltage_derivatives[-1]) * line[2])
    else:
      sensitivity+=(I[c]**3)*((voltage_derivatives[int(line[1]) - 1] - voltage_derivatives[int(line[0]) - 1]) * line[2])
    c+=1

  return sensitivity

In [ ]:
lines=np.array([[  2.   ,   3.   , 153.28 ],
       [  3.   ,   4.   ,  71.53 ],
       [  1.   ,   4.   ,  97.54 ],
       [  1.   ,   3.   , 119.224]])
Y_bus=get_ybus_matrix(lines)
Dnull=D[2]
i,j,k=3,4,1

p=[(get_sensitivity(3,2,3,4,2,D[0]),1),(get_sensitivity(5,2,3,1,2,D[1]),2),(get_sensitivity(3,5,3,4,1,D[2]),3),(get_sensitivity(4,3,4,1,3,D[3]),4),(get_sensitivity(5,4,1,3,4,D[4]),5)]

p.sort(key=lambda x:abs(x[0]))
print(p)
print(get_sensitivity(2,3,4,3,2,D[0]))
print(get_sensitivity(5,2,3,1,2,D[1]))
print(get_sensitivity(3,5,3,4,1,D[2]))
print(get_sensitivity(4,3,4,1,3,D[3]))
print(get_sensitivity(5,4,1,3,4,D[4]))

[(0.2865821159996172, 2), (-0.5808772435930829, 4), (-4.379689762695497, 1), (-4.800327012730994, 3), (9.035871408204576, 5)]
0.05490552093230905
0.2865821159996172
-4.800327012730994
-0.5808772435930829
9.035871408204576


# Security Based DC-OPF with IDC-PFC considered

In [ ]:
def create_adjacency_matrix(lines):
    num_buses = int(np.max(lines[:, :2]))
    adj = np.zeros(shape=(num_buses, num_buses))
    for line in lines:
        adj[int(line[0]) - 1][int(line[1]) - 1] = line[2]
        adj[int(line[1]) - 1][int(line[0]) - 1] = line[2]
    return adj

def get_ybus_matrix(lines):
    num_buses = int(np.max(lines[:, :2]))
    adj = create_adjacency_matrix(lines)
    y_bus = np.zeros(shape=(num_buses, num_buses))
    for i in range(num_buses):
        for j in range(num_buses):
            y_bus[i][j] = -1 * adj[i][j] if i != j else np.sum(adj[i])
    return y_bus

def power_balance(i,j,k,x):
    D=x[0]
    E=x[1]

    V_bus = [x[2:-len(on_shore)],x[-len(on_shore):]]

    P = np.diag(V_bus[0].reshape((-1,1))[:, 0]) @ Y_bus @ V_bus[0]

    power_onshore = np.zeros((num_buses,1))

    for i1 in range(len(on_shore)):
      bus=on_shore[i1]-1
      power_onshore[bus][0]=-kdroop[i1]*(V_bus[0][bus]-V_bus[1][i1])


    P_inj = np.array([0]*num_buses,dtype=np.float64)

    Pit = Y_bus[i-1][j-1]*(1-D)*V_bus[0][i-1]*E
    Piu = -Y_bus[i-1][k-1]*D*V_bus[0][i-1]*E
    Pj  = -Y_bus[i-1][j-1]*(1-D)*V_bus[0][j-1]*E
    Pk  = Y_bus[i-1][k-1]*D*V_bus[0][k-1]*E

    P_inj[i-1]+=Pit+Piu
    P_inj[j-1]+=Pj
    P_inj[k-1]+=Pk

    return P.flatten()-P_gen.flatten()-power_onshore.flatten()-P_inj.flatten()

def power_check(x):
    V_bus = [x[2:-len(on_shore)],x[-len(on_shore):]]
    power_onshore = np.zeros((num_buses,1))
    for i in range(len(on_shore)):
      bus=on_shore[i]-1
      power_onshore[bus][0]=-kdroop[i]*(V_bus[0][bus]-V_bus[1][i])
    return np.array([0,1.08695,1.08695,0])-np.abs(power_onshore.flatten())

def current_check(i,j,k,x):

    D=x[0]
    E=x[1]
    V_bus = [x[2:-len(on_shore)],x[-len(on_shore):]]
    I_max = np.array([1962] * lines.shape[0]) / base_current
    I=[]
    for line in lines:
      if(line[0]==i and line[1]==j):
        I.append((V_bus[0][int(line[0]) - 1] + (1-D)*E - V_bus[0][int(line[1]) - 1]) * line[2])
      elif(line[0]==j and line[1]==i):
        I.append((V_bus[0][int(line[1]) - 1] + (1-D)*E - V_bus[0][int(line[0]) - 1]) * line[2])
      elif(line[0]==i and line[1]==k):
        I.append((V_bus[0][int(line[0]) - 1] + (-D*E) - V_bus[0][int(line[1]) - 1]) * line[2])
      elif(line[0]==k and line[1]==i):
        I.append((V_bus[0][int(line[1]) - 1] + (-D*E) - V_bus[0][int(line[0]) - 1]) * line[2])
      else:
        I.append((V_bus[0][int(line[0]) - 1]- V_bus[0][int(line[1]) - 1]) * line[2])

    I = np.array(I)
    return I_max-np.abs(I)

def power_flow_controller(i,j,k,x):
    D = x[0]
    E = x[1]
   # print(E-((D*Y_bus[i-1][k-1]-(1-D)*Y_bus[i-1][j-1])*x[i+1]  +  (1-D)*Y_bus[i-1][j-1]*x[j+1]  - D*Y_bus[i-1][k-1]*x[k+1])/(((1-D)**2)*Y_bus[i-1][j-1]+(D**2)*Y_bus[i-1][k-1]))
    return E-((D*Y_bus[i-1][k-1]-(1-D)*Y_bus[i-1][j-1])*x[i+1]  +  (1-D)*Y_bus[i-1][j-1]*x[j+1]  - D*Y_bus[i-1][k-1]*x[k+1])/(((1-D)**2)*Y_bus[i-1][j-1]+(D**2)*Y_bus[i-1][k-1])

def performance_index(i,j,k,x):
    # i-->j 3-->4
    # i-->k 3-->1

    D=x[0]
    E=x[1]
    V_bus = [x[2:-len(on_shore)],x[-len(on_shore):]]
    I_max = np.array([1962] * lines.shape[0]) / base_current
    I=[]

    for line in lines:
      if(line[0]==i and line[1]==j):
        I.append((V_bus[0][int(line[0]) - 1] + (1-D)*E - V_bus[0][int(line[1]) - 1]) * line[2])
      elif(line[0]==j and line[1]==i):
        I.append((V_bus[0][int(line[1]) - 1] + (1-D)*E - V_bus[0][int(line[0]) - 1]) * line[2])
      elif(line[0]==i and line[1]==k):
        I.append((V_bus[0][int(line[0]) - 1] + (-D*E) - V_bus[0][int(line[1]) - 1]) * line[2])
      elif(line[0]==k and line[1]==i):
        I.append((V_bus[0][int(line[1]) - 1] + (-D*E) - V_bus[0][int(line[0]) - 1]) * line[2])
      else:
        I.append((V_bus[0][int(line[0]) - 1]- V_bus[0][int(line[1]) - 1]) * line[2])

    I = np.array(I)
    I_norm = I / I_max
    return np.sum(I_norm**4)/ 4


def get_idcpfc_opf_results(i,j,k,lines):
    condition1 = {'type': 'eq', 'fun': lambda x : power_balance(i,j,k,x).flatten()}
    condition2 = {'type': 'ineq', 'fun': lambda x : current_check(i,j,k,x).flatten()}
    condition3 = {'type': 'ineq', 'fun': lambda x : power_check(x).flatten()}
    condition4 = {'type': 'eq', 'fun': lambda x: power_flow_controller(i,j,k,x).flatten()}

    conditions = [condition1, condition2,condition3,condition4]

    global Y_bus
    Y_bus = get_ybus_matrix(lines)

    lower_bounds=[V_min]*(num_buses+len(on_shore)+2)
    lower_bounds[0]=0
    lower_bounds[1]=-0.05

    upper_bounds=[V_max]*(num_buses+len(on_shore)+2)
    upper_bounds[0]=1
    upper_bounds[1]=0.05

    bounds=Bounds(lb=lower_bounds, ub=upper_bounds)

    x0 = np.array([1.05 for i in range(num_buses+len(on_shore)+2)])
    x0[0]=0.5
    x0[1]=0.01

    sol = minimize(lambda x : performance_index(i,j,k,x), x0, method='trust-constr', bounds=bounds,constraints=conditions, options={'maxiter':2000})

    V_bus = np.array(sol.x[:-1*len(on_shore)])

    return sol

In [ ]:
interline = {(2,3):(3,2,4),
             (2,5):(3,2,1),
             (3,5):(3,4,1),
             (3,4):(4,3,1),
             (4,5):(1,3,4)}


i,j,k = interline[(3,5)]

lines = np.array([
                  [2, 3, 153.28],
                  [3, 4, 71.53],
                  [4, 1, 97.54],
                  [1, 3, 119.224]
                  ])

get_idcpfc_opf_results(3,4,1,lines)

           message: `gtol` termination condition is satisfied.
           success: True
            status: 1
               fun: 0.5464603827289944
                 x: [ 4.910e-01 -9.573e-03  1.050e+00  1.032e+00  1.037e+00
                      1.045e+00  9.913e-01  9.872e-01]
               nit: 74
              nfev: 594
              njev: 66
              nhev: 0
          cg_niter: 91
      cg_stop_cond: 4
              grad: [-1.582e+00  2.328e+01  1.165e+02 -9.393e+01 -7.128e+01
                      4.876e+01  0.000e+00  0.000e+00]
   lagrangian_grad: [ 7.675e-09 -4.748e-10  4.663e-14 -1.840e-10 -1.849e-10
                     -1.490e-10 -1.712e-10 -1.848e-10]
            constr: [array([-4.210e-11, -1.009e-11, -8.796e-11, -1.234e-10]), array([ 1.506e-01,  6.835e-02,  5.467e-01,  3.430e-02]), array([ 0.000e+00,  2.107e-01,  5.504e-07,  0.000e+00]), array([ 2.088e-12]), array([ 4.910e-01, -9.573e-03,  1.050e+00,  1.032e+00,
                            1.037e+00,  1.045e+00,  9

In [ ]:
x=get_idcpfc_opf_results(3,4,1,lines).x
I=[0]*num_lines
c=1
D=x[0]
E=x[1]
V_bus = [x[2:-len(on_shore)],x[-len(on_shore):]]
I_max = np.array([1962] * lines.shape[0]) / base_current
I=[]

for line in lines:
      if(line[0]==i and line[1]==j):
        I.append((V_bus[0][int(line[0]) - 1] + (1-D)*E - V_bus[0][int(line[1]) - 1]) * line[2])
      elif(line[0]==j and line[1]==i):
        I.append((V_bus[0][int(line[1]) - 1] + (1-D)*E - V_bus[0][int(line[0]) - 1]) * line[2])
      elif(line[0]==i and line[1]==k):
        I.append((V_bus[0][int(line[0]) - 1] + (-D*E) - V_bus[0][int(line[1]) - 1]) * line[2])
      elif(line[0]==k and line[1]==i):
        I.append((V_bus[0][int(line[1]) - 1] + (-D*E) - V_bus[0][int(line[0]) - 1]) * line[2])
      else:
        I.append((V_bus[0][int(line[0]) - 1]- V_bus[0][int(line[1]) - 1]) * line[2])

I = np.array(I)

In [ ]:
V_bus

[array([1.04999984, 1.03165812, 1.03719955, 1.04535204]),
 array([0.99134899, 0.98719957])]

In [ ]:
I

array([-0.84938999, -0.93165481, -0.45334704, -0.96570079])

In [ ]:
D,E

(0.4910280437254315, -0.009572592858083522)

In [ ]:
import numpy as np
from scipy.optimize import root

# Define the system of equations
def system_equations(x):
    eq1 = x-np.exp(x)

    return eq1

# Initial guess for the solution vector x
initial_guess = np.array([0.5, 0.5])

# Use the Newton-Raphson method to solve the system of equations
result = root(system_equations, initial_guess, method='lm')

# Get the final solution
solution = result.x

print("Final Solution for x:", solution)


Final Solution for x: [-1.08359017e-06 -1.08359017e-06]
